# 3.3 线性回归的简洁实现

In [2]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)        #随机种子

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')
  #这个方法的意思是设置pytorch中默认的浮点类型，一般使用pytorch进行运算时候使用的都是浮点数来进行计算，所以设置默认浮点数有时候也很重要。
  #默认的Tensor是FloatTensor，默认CPU计算

1.8.1+cu111


## 3.3.1 生成数据集

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [4]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)      
#用来对 tensor 进行打包，就好像 python 中的 zip 功能。该类通过每一个 tensor 的第一个维度进行索引。因此，该类中的 tensor 第一维度必须相等。

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [16]:
#测试数据打包

from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader

a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = torch.tensor([44, 55, 66, 44, 55, 66, 44, 55, 66, 44, 55, 66])
train_ids = TensorDataset(a, b)         #打包
# 切片输出
print(train_ids[0:2])
print('=' * 80)
# 循环取数据
for x_train, y_label in train_ids:
    print(x_train, y_label)
# DataLoader进行数据封装
print('=' * 80)
train_loader = DataLoader(dataset=train_ids, batch_size=4, shuffle=True)        #四个为一组，带标签的
for i, data in enumerate(train_loader, 1):  # 注意enumerate返回值有两个,一个是序号，一个是数据（包含训练数据和标签）
    x_data, label = data
    print(' batch:{0} x_data:{1}  label: {2}'.format(i, x_data, label))



(tensor([[1, 2, 3],
        [4, 5, 6]]), tensor([44, 55]))
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
 batch:1 x_data:tensor([[4, 5, 6],
        [7, 8, 9],
        [4, 5, 6],
        [1, 2, 3]])  label: tensor([55, 66, 55, 44])
 batch:2 x_data:tensor([[7, 8, 9],
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])  label: tensor([66, 44, 55, 66])
 batch:3 x_data:tensor([[7, 8, 9],
        [4, 5, 6],
        [1, 2, 3],
        [1, 2, 3]])  label: tensor([66, 55, 44, 44])


In [5]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.8692,  0.3709],
        [ 0.2087, -1.1668],
        [-0.5374, -0.1184],
        [-0.8751,  0.8386],
        [ 0.7958,  0.2482],
        [-2.3310, -0.5360],
        [-1.4001, -0.5942],
        [ 0.3616,  0.0934],
        [ 0.0121,  1.9686],
        [-0.1894, -0.9543]]) 
 tensor([ 4.6863,  8.5955,  3.5257, -0.3987,  4.9537,  1.3598,  3.4271,  4.5885,
        -2.4839,  7.0571])


## 3.3.3 定义模型

In [6]:
class LinearNet(nn.Module):     #定义网络
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.6873, 0.2890]], requires_grad=True)
Parameter containing:
tensor([-0.1508], requires_grad=True)


## 3.3.4 初始化模型参数

In [9]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)    #用值val填充向量。初始化bias为0e
# 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [10]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0100, -0.0072]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [11]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [12]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [14]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
i_list=[i for i in optimizer.param_groups[0].keys()]
print(i_list) 

['params', 'lr', 'momentum', 'dampening', 'weight_decay', 'nesterov']


## 3.3.7 训练模型

In [15]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 27.492670
epoch 2, loss: 22.722670
epoch 3, loss: 22.273788


In [15]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 1.9999, -3.4005]])
4.2 tensor([4.2011])
